### Import spin up files, plot KE and PV evolution

In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

In [ ]:
# # import spin-up files
ds = xr.open_mfdataset('/burg/abernathey/users/hillary/spin_up/*.nc')
ds

In [ ]:
# Mean Eddy Kinetic Energy (EKE) as a function of time
dEKE_dt = np.gradient(ds.EKE.isel(lev=0), ds.time)
I = int(np.where(np.abs(ds.time - year*14)==np.min(np.abs(ds.time - year*14)))[0])
time_stable = ds.time[I].values

# Plot EKE derivative
plt.rcParams.update({'font.size': 18})
plt.figure(figsize=(12,6)); plt.plot(ds.time/year, dEKE_dt)
plt.ylabel('EKE central differece'); plt.xlabel('model time (year)')
plt.axvline(x=time_stable/year, color='k', linestyle='--')


print('model appears stable at', ds.time[I].values/year, 'years');

In [ ]:
# Plot time series of EKE
plt.figure(figsize=(12,6))
plt.semilogy(ds.time/year, ds.EKE.isel(lev=0), lw=4, color='purple', label='upper layer')
plt.semilogy(ds.time/year, ds.EKE.isel(lev=1), lw=4, color='darkorange', label='lower layer')
ints = [10000, 15000, 25000, 35000, 55000, 70000]
for i in ints:
    plt.axvline(x=ds.time[i]/year, color='lightblue', linewidth=2, alpha=1)
plt.axvline(x=time_stable/year, color='k', lw=2, linestyle='--')
plt.grid(True, alpha=0.5); plt.legend(frameon=False, ncol=2)
plt.ylabel(r'KE ($\rmcm^{2}$/$\rms^{2}$)'); plt.xlabel('Model Time (year)');

As the model gets spun up, the mean EKE increases until it reaches an equilibrated state. When the EKE plateaus the model is in a stable state. The dashed vertical line marks 12.5 years when the model appears stable. This would be a good place to save the model state for the ensemble perturbation experiment.

Vertical light blue lines show times corresponding the the snapshots plotted below. 

In [ ]:
# Evolution of the upper PV anomaly field
print('stable index ', int(np.where(ds.time == time_stable)[0]))

plt.rcParams.update({'font.size': 10})
plt.rcParams['image.cmap'] = 'RdBu'

ints = [0, 1000, 5000, 10000, 20000, 50000]
plt.figure(figsize=(15,15))
for i in enumerate(ints):
    plt.subplot(2,3,i[0]+1)
    plt.pcolormesh(ds.q[i[1],0,:,:])
    plt.title((ds.time[i[1]].values/year).round(decimals=2), color='k') # The titles correspond to the model time in years. 
    plt.colorbar()

In [ ]:
# Save model state at equilibrium
I = int(np.where(np.abs(ds.time - time_stable)==np.min(np.abs(ds.time - time_stable)))[0])
qg_equilibrium = ds[dict(time=I)]

path = '/burg/abernathey/users/hillary/'
qg_equilibrium.to_netcdf(path+'QG_equilibrium_proto.nc', engine='h5netcdf', invalid_netcdf=True)